<a href="https://colab.research.google.com/github/profshai/Python-MachineLearning-DeepLearning-Projects/blob/master/Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation with Neural Networks

In this notebook we will create a network that can generate text, here we show it being done character by character. Very awesome write up on this here: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

We organized the process into "steps" so you can easily follow along with your own data sets.

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [30]:
tf.__version__

'2.2.0-rc3'

## Step 1: The Data

The data can be obtained from here: https://www.gutenberg.org/

It is based on shakespeare's works, mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [0]:
path_to_file = 'drive/My Drive/udemytfdata/06-NLP-and-Text-Data/shakespeare.txt'

In [0]:
text = open(path_to_file, 'r').read()

In [33]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [34]:
print(text[140500:141000])

reward.
  HELENA. Inspired merit so by breath is barr'd.
    It is not so with Him that all things knows,
    As 'tis with us that square our guess by shows;
    But most it is presumption in us when
    The help of heaven we count the act of men.
    Dear sir, to my endeavours give consent;
    Of heaven, not me, make an experiment.
    I am not an impostor, that proclaim  
    Myself against the level of mine aim;
    But know I think, and think I know most sure,
    My art is not past power n


### Understanding unique characters

In [35]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [0]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [37]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [0]:
ind_to_char = np.array(vocab)

In [39]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [0]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [41]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [42]:
encoded_text.shape

(5445609,)

We now have a mapping we can use to go back and forth from characters to numerics.

In [43]:
sample = text[:20]
sample

'\n                   '

In [44]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [45]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [0]:
line = "From fairest creatures we desire increase"

In [47]:
len(line)

41

In [0]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [49]:
len(part_stanza)

131

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [0]:
#we will choose a sequence length of 120
seq_len = 120

In [0]:
total_num_seq = len(text)//(seq_len+1)

In [52]:
total_num_seq

45005

In [0]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

#for i in char_dataset.take(500):
    # print(ind_to_char[i.numpy()])

The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [0]:
sequences=char_dataset.batch(seq_len+1, drop_remainder=True) #division is not exact

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [0]:
def create_seq_targets(seq):
    input_txt = seq[:-1] #Hello my nam
    target_txt = seq[1:] #ello my name
    return input_txt, target_txt

In [0]:
dataset = sequences.map(create_seq_targets)

In [57]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [0]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [59]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Step 4: Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True. This is because of the one hot encoding

In [0]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [63]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [0]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [0]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [0]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [67]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.

In [68]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [69]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 6.0001276e-03,  2.7819953e-03,  2.5198890e-03, ...,
          1.3863188e-03,  2.3697233e-03, -1.1174754e-03],
        [ 2.2084436e-03, -2.4858688e-03, -1.7595991e-03, ...,
          2.3008662e-03,  4.2997282e-03, -1.4557852e-03],
        [ 1.9140893e-03, -2.0804496e-03,  5.9937302e-04, ...,
         -4.5144726e-03,  4.4333925e-03, -2.1281745e-03],
        ...,
        [-4.5853835e-03,  5.4905289e-03,  2.4673005e-03, ...,
         -5.1275804e-03, -2.5460527e-03,  1.2396579e-03],
        [-3.1759234e-03,  3.5066484e-04,  7.0637604e-04, ...,
         -2.3460388e-03, -2.3086409e-03,  3.7663097e-03],
        [ 2.0723334e-03,  5.8374749e-03,  1.4644928e-03, ...,
         -5.9825066e-03,  1.2929252e-03, -4.3711928e-03]],

       [[-1.8723856e-03, -4.5685433e-03, -2.4194291e-03, ...,
          1.6433421e-03,  2.9641192e-03, -1.5931553e-04],
        [-2.0787769e-04,  2.1628544e-03, -1.2493427e-03, ...,
         -2.9464015e-03,  1

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [71]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[20],
       [55],
       [42],
       [79],
       [71],
       [75],
       [39],
       [40],
       [ 7],
       [64],
       [50],
       [53],
       [ 0],
       [44],
       [30],
       [10],
       [71],
       [ 4],
       [29],
       [ 8],
       [13],
       [18],
       [83],
       [56],
       [49],
       [25],
       [ 5],
       [ 7],
       [78],
       [82],
       [19],
       [30],
       [69],
       [46],
       [39],
       [ 5],
       [39],
       [35],
       [11],
       [63],
       [62],
       [75],
       [27],
       [55],
       [44],
       [46],
       [32],
       [23],
       [36],
       [48],
       [50],
       [ 7],
       [ 3],
       [ 4],
       [63],
       [34],
       [50],
       [49],
       [14],
       [13],
       [ 4],
       [48],
       [31],
       [ 8],
       [39],
       [83],
       [65],
       [49],
       [22],
       [83],
       [41],
       [43],
       [ 7],
   

In [0]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [73]:
sampled_indices

array([20, 55, 42, 79, 71, 75, 39, 40,  7, 64, 50, 53,  0, 44, 30, 10, 71,
        4, 29,  8, 13, 18, 83, 56, 49, 25,  5,  7, 78, 82, 19, 30, 69, 46,
       39,  5, 39, 35, 11, 63, 62, 75, 27, 55, 44, 46, 32, 23, 36, 48, 50,
        7,  3,  4, 63, 34, 50, 49, 14, 13,  4, 48, 31,  8, 39, 83, 65, 49,
       22, 83, 41, 43,  7,  8, 59, 44, 13, 71, 30, 23, 11, 77, 35, 36, 49,
       55, 65, 16, 62, 55, 59, 46, 56,  4, 73, 30, 18, 76, 35, 22, 28, 66,
        4, 46, 80, 41, 13, 70, 83, 36, 39, 12, 53, 18,  3,  2, 62, 20, 40,
       64])

In [74]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))

Given the input seq: 

s most foul, strange, and unnatural.
  Ham. Haste me to know't, that I, with wings as swift
    As meditation or the tho


Next Char Predictions: 

9`QxptNO)iY]
SE.p&D,27}aX?')w|8EnUN'NJ0hgtB`SUG<KWY)"&hIYX32&WF,N}jX;}PR),dS2pE<0vJKX`j5g`dUa&rE7uJ;Ck&UyP2o}KN1]7"!g9Oi


After confirming the dimensions are working, let's train our network!

In [0]:
epochs = 30

In [76]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
351/351 [==============================] - 2111s 6s/step - loss: 2.5082
Epoch 2/30
351/351 [==============================] - 2063s 6s/step - loss: 1.7045
Epoch 3/30
351/351 [==============================] - 2100s 6s/step - loss: 1.4469
Epoch 4/30
351/351 [==============================] - 2069s 6s/step - loss: 1.3330
Epoch 5/30
351/351 [==============================] - 2090s 6s/step - loss: 1.2717
Epoch 6/30
351/351 [==============================] - 2086s 6s/step - loss: 1.2326
Epoch 7/30
351/351 [==============================] - 2066s 6s/step - loss: 1.2033
Epoch 8/30
351/351 [==============================] - 2091s 6s/step - loss: 1.1799
Epoch 9/30
351/351 [==============================] - 2090s 6s/step - loss: 1.1600
Epoch 10/30
351/351 [==============================] - 2112s 6s/step - loss: 1.1429
Epoch 11/30
351/351 [==============================] - 2125s 6s/step - loss: 1.1267
Epoch 12/30
351/351 [==============================] - 2071s 6s/step - loss: 1.1128
E

KeyboardInterrupt: ignored

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [0]:
model.save('shakespeare_gen.h5') 

In [0]:
from tensorflow.keras.models import load_model

In [0]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [80]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [82]:
print(generate_text(model,"flower",gen_size=1000))

flower it in a squeeve for many a thip for fashion, far,
    'Tis I assail you, he deserves as me,
    And, take my mind to sour well in
    fellow to the Queen.
  AUTOLYCUS. Then should estalligh fell. Call till the keen idleness.
    Now, anw-
  PRINCE JENUS. What's he cow?
  SEEPTON. All things arrated,
    If she banisherous hands she craves of broken,
    Our gates and talk we owe theperhwives in and
    laudives him home again.
  HELENA. I am not born of all Love's unnatural and to come
    both.
DROMIO OF EPHESUS. My lord.
  Ham. The King, his Grace put yourselves to car sail  
    The tenour of thine own not to mine?
  PAROLLES. Marry, that remove it is, his grace I have lodged all,
    And thereby love doth raise us render draw.
  HASTINGS. O Leanceloous, imperish for twenty,
    To lead Athous PLET   llow.
    Accurs'd thee turns?     [She over him whot must be look'd upon a musterer; and for him
    would be a cut he hath got very sweet and learn'd down.
  AUFIDIUS. My large